In [1]:
import time
import math
import re
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from pandas_ods_reader import read_ods
from  matplotlib.ticker import PercentFormatter
import seaborn as sns
 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
 
from sklearn import set_config
# set_config(display='diagram') # Để trực quan hóa pipeline
 
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.utils import resample

## Combine

In [209]:
df_1 = pd.read_excel('data/RTAs_2012_2018_Time_Series.xlsx')

In [210]:
df_2 = pd.read_excel('data/Road Safety Data - Accidents 2019.xlsx')

In [211]:
df_1.shape

(959847, 32)

In [212]:
df_1 = df_1.append(df_2,ignore_index = True)

In [242]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077383 entries, 0 to 1077382
Data columns (total 32 columns):
 #   Column                                       Non-Null Count    Dtype  
---  ------                                       --------------    -----  
 0   Accident_Index                               1077383 non-null  object 
 1   Location_Easting_OSGR                        1077247 non-null  float64
 2   Location_Northing_OSGR                       1077247 non-null  float64
 3   Longitude                                    1077237 non-null  float64
 4   Latitude                                     1077237 non-null  float64
 5   Police_Force                                 1077383 non-null  int64  
 6   Accident_Severity                            1077383 non-null  int64  
 7   Number_of_Vehicles                           1077383 non-null  int64  
 8   Number_of_Casualties                         1077383 non-null  int64  
 9   Date                                         1

In [220]:
df_1['Date'] 

0          2012-01-01 00:00:00
1          2012-01-01 00:00:00
2          2012-01-01 00:00:00
3          2012-01-01 00:00:00
4          2012-01-01 00:00:00
                  ...         
1077378             31/12/2019
1077379             31/12/2019
1077380             31/12/2019
1077381             31/12/2019
1077382             31/12/2019
Name: Date, Length: 1077383, dtype: object

In [237]:
# for i in df_1['Date']:
#     print(str(i))

In [241]:
df_1['Time'][df_1['Date'] == "2012-01-04 00:00:00"]

Series([], Name: Time, dtype: object)

In [238]:
# x['Time'][x['Date'] == '2012-01-04 00:00:00']
for ii in df_1['Time'][df_1['Date'] == '2012-01-04 00:00:00']:
    if ii == '17:00:00':
        print(ii)
# x.shape

In [6]:
df_1.to_csv('data/RTAs_2012_2019_Time_Series.csv',index=False)

## Preprocessing

https://roadtraffic.dft.gov.uk/api/local-authorities/

In [44]:
df = pd.read_csv('data/RTAs_2012_2019_Time_Series.csv')
df.shape

(1077383, 32)

In [45]:
df.columns

Index(['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR',
       'Longitude', 'Latitude', 'Police_Force', 'Accident_Severity',
       'Number_of_Vehicles', 'Number_of_Casualties', 'Date', 'Day_of_Week',
       'Time', 'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'LSOA_of_Accident_Location'],
      dtype='object')

Ta bỏ bớt một số cột không cần dùng trong đồ án này:

'Accident_Index','Location_Easting_OSGR', 'Location_Northing_OSGR','Longitude', 'Latitude', 'Police_Force', 'Local_Authority_(District)', '1st_Road_Number', '2nd_Road_Number', 'Did_Police_Officer_Attend_Scene_of_Accident', 'LSOA_of_Accident_Location'

In [47]:
df['Date'].unique().shape

(2922,)

In [48]:
for date in df['Date'].unique():
    if len(date) != 10:
        df['Date'][df['Date'] == date] = date[0:10]


<ipython-input-48-e54b3d48f40f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'][df['Date'] == date] = date[0:10]


In [56]:
df['Date']

0          2012-01-01
1          2012-01-01
2          2012-01-01
3          2012-01-01
4          2012-01-01
              ...    
1077378    31/12/2019
1077379    31/12/2019
1077380    31/12/2019
1077381    31/12/2019
1077382    31/12/2019
Name: Date, Length: 1077383, dtype: object

In [100]:
# x = df.copy()
df = x.copy()
df.shape

(1077383, 32)

In [101]:
# df = df.dropna(subset=['Time'])
df = df.dropna(subset=['Time']).reset_index(drop=True)

In [107]:
df.shape

(1077271, 32)

In [103]:
df['Date'] = df['Date'] + ' ' + df['Time']

In [159]:
pd.to_datetime(['12/03/2012 17:00', '12-03-2012 19:00'])

DatetimeIndex(['2012-12-03 17:00:00', '2012-12-03 19:00:00'], dtype='datetime64[ns]', freq=None)

In [105]:
df['Date'] = pd.to_datetime(df['Date'])

In [163]:
df['Date'][df['Date'] == '2012-01-04 17:00:00']

1352   2012-01-04 17:00:00
1353   2012-01-04 17:00:00
1354   2012-01-04 17:00:00
1355   2012-01-04 17:00:00
Name: Date, dtype: datetime64[ns]

In [200]:
x = pd.read_csv('data/RTAs_2012_2019_Time_Series.csv')

In [206]:
x['Date']

0          2012-01-01 00:00:00
1          2012-01-01 00:00:00
2          2012-01-01 00:00:00
3          2012-01-01 00:00:00
4          2012-01-01 00:00:00
                  ...         
1077378             31/12/2019
1077379             31/12/2019
1077380             31/12/2019
1077381             31/12/2019
1077382             31/12/2019
Name: Date, Length: 1077383, dtype: object

In [208]:
# x['Time'][x['Date'] == '2012-01-04 00:00:00']
for ii in x['Time'][x['Date'] == '2012-01-04 00:00:00']:
    if ii == '17:00:00':
        print(ii)
x.shape

17:00:00
17:00:00
17:00:00
17:00:00


(1077383, 32)

In [192]:
for date in x['Date'].unique():
    if len(date) != 10:
        x['Date'] = x['Date'].replace(date, date[0:10])


In [198]:
x['Time'][x['Date'] == '2012-01-04']

1112    00:01:00
1113    00:10:00
1114    00:29:00
1115    00:31:00
1116    00:32:00
          ...   
1436    22:50:00
1437    22:53:00
1438    22:56:00
1439    23:07:00
1440    23:50:00
Name: Time, Length: 329, dtype: object

In [199]:
for ii in x['Time'][x['Date'] == '2012-01-04']:
    if ii == '17:00:00':
        print(ii)
x.shape

17:00:00
17:00:00
17:00:00
17:00:00


(1077383, 32)

In [106]:
# y = df.copy()
df = y.copy()

In [108]:
df = df.sort_values('Date').reset_index(drop=True)
df.shape

(1077271, 32)

In [125]:
import requests
import json
try:
    response = requests.get("https://roadtraffic.dft.gov.uk/api/local-authorities")
    json_pydata = json.loads(response.text)
except Exception as error:
     print(error)
data_json = []
if json_pydata['data'] != None:
    data_json.extend(json_pydata['data'])

In [139]:
for data in data_json:
    if data['ons_code'] == 'E08000005':
        print(data['region']['name'])
# data_json[0]#['ons_code']
# data_json[0]['region']['name']
# 'E08000005'

North West


In [146]:
df['region'] = df['Local_Authority_(Highway)']

In [148]:
df['region'].value_counts()

E10000016    33277
E10000030    28901
E10000012    23052
E10000014    22492
E10000017    22431
             ...  
EHEATHROW      294
S12000013      206
S12000027      178
S12000023      149
E06000053       13
Name: region, Length: 207, dtype: int64

In [157]:
df['Date'][0].year

2012

In [153]:
for local in df['region'].unique():
    if local in ['S12000043', 'S12000015', 'S12000024', 'S12000009']:
        df['region'] = df['region'].replace(local, 'Scotland')
    elif local in ['E06000048', 'E08000020']:
        df['region'] = df['region'].replace(local, 'North East')
    elif local in ['EHEATHROW']:
        df['region'] = df['region'].replace(local, 'London')
    else:
        for data in data_json:
            if data['ons_code'] == local:
                df['region'] = df['region'].replace(local, data['region']['name'])
                break

In [154]:
df['region'].value_counts()

London                      201634
South East                  171294
North West                  107442
East of England             102614
Yorkshire and The Humber     95438
West Midlands                91126
South West                   85787
East Midlands                79609
Scotland                     63429
Wales                        41292
North East                   37606
Name: region, dtype: int64

## Xử lý thời gian

Ta chuyển 2 cột `Date` và `Time` về thành một cột.

Với dataset dùng để huấn luyện, ta tách cột này ra khỏi data

## Xử lý kiểu dữ liệu cột

Hiện tại một số cột đang có dạng số, nhưng thực ra là dạng categorical, ta chuyển lại về chữ nhờ file `variable_lookup.xls` do chính phủ Anh cung cấp để dễ dàng hiểu dữ liệu và tìm insight hơn.

1. Các cột categorical không có thứ tự, ta chuyển lại về chữ cho cả dataframe dùng để tìm insight, lẫn dataframe dùng để huấn luyện model:

'Road_Type', 'Junction_Detail', 'Junction_Control', 'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities', 'Special_Conditions_at_Site', 'Carriageway_Hazards', 'Local_Authority_(Highway)'

2. Các cột dạng categorical có thứ tự: ta chuyển lại về chữ cho dataframe dùng để tìm insight, nhưng giữ nguyên cho dataframe dùng để huấn luyện model:

'Accident_Severity', 'Day_of_Week', '1st_Road_Class', '2nd_Road_Class', 'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 'Urban_or_Rural_Area'

3. Đặc biệt, ta thấy cột 'Local_Authority_(Highway)' có đến 208 giá trị, ta sẽ chuyển các vùng địa phương này thành khu vực (region) nó phụ thuộc vào, để nghiến cứu trên phạm vi lớn hơn, ta có thể chuyển nhờ file API sau:

https://roadtraffic.dft.gov.uk/api/local-authorities1

Một số ons_code không có trong api này, ta sẽ tra file variable_lookup.xls và chuyển thủ công





In [246]:
unordered_cate_cols = ['Road_Type', 'Junction_Detail', 
                'Junction_Control', 'Pedestrian_Crossing-Human_Control', 
                'Pedestrian_Crossing-Physical_Facilities',
                'Special_Conditions_at_Site', 'Carriageway_Hazards',
                      'Local_Authority_(Highway)']
        
ordered_cate_cols = ['Accident_Severity', 'Day_of_Week', '1st_Road_Class',
                             '2nd_Road_Class', 'Light_Conditions', 'Weather_Conditions',
                             'Road_Surface_Conditions', 'Urban_or_Rural_Area']

nume_cols = ['Number_of_Vehicles', 'Number_of_Casualties', 'Speed_limit']

In [247]:
unordered_cate_cols += ['region']

unordered_cate_cols

['Road_Type',
 'Junction_Detail',
 'Junction_Control',
 'Pedestrian_Crossing-Human_Control',
 'Pedestrian_Crossing-Physical_Facilities',
 'Special_Conditions_at_Site',
 'Carriageway_Hazards',
 'Local_Authority_(Highway)',
 'region']

In [178]:
df = df.drop(['Accident_Index','Location_Easting_OSGR', 'Location_Northing_OSGR','Longitude', 
                      'Latitude', 'Police_Force', 'Local_Authority_(District)', '1st_Road_Number', 
                      '2nd_Road_Number', 'Did_Police_Officer_Attend_Scene_of_Accident',
                      'LSOA_of_Accident_Location'],1)

In [180]:
df.columns

Index(['Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties',
       'Date', 'Day_of_Week', 'Time', 'Local_Authority_(Highway)',
       '1st_Road_Class', 'Road_Type', 'Speed_limit', 'Junction_Detail',
       'Junction_Control', '2nd_Road_Class',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'region'],
      dtype='object')

In [182]:
len(unordered_cate_cols + ordered_cate_cols + nume_cols)

19

In [173]:
class ColAdjustBeforeSplitData(BaseEstimator, TransformerMixin):
    def __init__(self, mode = 'insight'):
#         mode = `insight` or `trainning`
        self.mode = mode
        pass
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        tf_df = X_df.copy()
        
        # Bước 1. Bỏ giá trị trùng
        
        tf_df = tf_df.drop_duplicates(ignore_index = True)
        
        #Bước 2. Bỏ cột
        
        tf_df = tf_df.drop(['Accident_Index','Location_Easting_OSGR', 'Location_Northing_OSGR','Longitude', 
                      'Latitude', 'Police_Force', 'Local_Authority_(District)', '1st_Road_Number', 
                      '2nd_Road_Number', 'Did_Police_Officer_Attend_Scene_of_Accident',
                      'LSOA_of_Accident_Location'],1)
        
        
        
        # Bước 3. Xửa lý datetime
        for date in tf_df['Date'].unique():
            if len(date) != 10:
                tf_df['Date'] = tf_df['Date'].replace(date, date[0:10])
                
        tf_df = tf_df.dropna(subset=['Time'])
        tf_df['Date'] = tf_df['Date'] + ' ' + tf_df['Time']
        tf_df['Date'] = pd.to_datetime(tf_df['Date'])
        tf_df = tf_df.sort_values('Date').reset_index(drop=True)

        tf_df = tf_df.drop(['Time'], 1)      
        # Bước 4. Map
        # 4.1  Region
        # get data
        try:
            response = requests.get("https://roadtraffic.dft.gov.uk/api/local-authorities")
            json_pydata = json.loads(response.text)
        except Exception as error:
             print(error)
        data_json = []
        if json_pydata['data'] != None:
            data_json.extend(json_pydata['data'])
            
        # create region column
        tf_df['region'] = tf_df['Local_Authority_(Highway)']
        
        # map local to region
        for local in tf_df['region'].unique():
            if local in ['S12000043', 'S12000015', 'S12000024', 'S12000009']:
                tf_df['region'] = tf_df['region'].replace(local, 'Scotland')
            elif local in ['E06000048', 'E08000020']:
                tf_df['region'] = tf_df['region'].replace(local, 'North East')
            elif local in ['EHEATHROW']:
                tf_df['region'] = tf_df['region'].replace(local, 'London')
            else:
                for data in data_json:
                    if data['ons_code'] == local:
                        tf_df['region'] = tf_df['region'].replace(local, data['region']['name'])
                        break
        
        # 4.2 map
        if self.mode == 'trainning' or self.mode == 'insight':
            
            for col in unordered_cate_cols:
                if col == 'Pedestrian_Crossing-Human_Control':
                    name = 'Ped Cross - Human'
                elif col == 'Pedestrian_Crossing-Physical_Facilities':
                    name = 'Ped Cross - Physical'
                else:
                    name = col.replace('_',' ')
                    
                map_df = pd.read_excel('/content/variable_lookup.xlsx',name)
                for code in map_df['code']:
                    tf_df[col] = tf_df[col].replace(code,map_df[map_df['code'] == code]['label'].values[0])
                    
        if self.mode == 'insight':
            for col in ordered_cate_cols:
                if col == 'Weather_Conditions':
                    name = 'Weather'
                elif col == 'Road_Surface_Conditions':
                    name = 'Road Surface'
                elif col == 'Urban_or_Rural_Area':
                    name = 'Urban Rural'
                else:
                    name = col.replace('_',' ')
                    
                map_df = pd.read_excel('/content/variable_lookup.xlsx',name)
                for code in map_df['code']:
                    tf_df[col] = tf_df[col].replace(code,map_df[map_df['code'] == code]['label'].values[0])
                    
        # Bước 5. pop date time
        
        if self.mode == 'trainning':
            date_time = tf_df.pop('Date')
            return (date_time, tf_df)
        else:
            return tf_df

In [16]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [17]:
contigency_df = pd.crosstab(df_1['Local_Authority_(District)'], df_1['Accident_Severity'])
contigency_df.columns = ['1','2','3']

NameError: name 'df_1' is not defined

In [ ]:
print((sum(contigency_df['1']) + sum(contigency_df['2']) + sum(contigency_df['3'])) == df_1.shape[0])
contigency_df
# df_1.shape
# contigency_df.xs('Accident_Severity', level='1', axis=1)
# items_df_preprocessed[df_1['Region'] == f'{region}'].groupby('Accident severity')['Accidents'].agg(sum).reset_index(name='Value count')

In [ ]:
Region_Severity_df = pd.DataFrame({'Region' : [np.nan],'Fatal' : [np.nan],'Serious' : [np.nan],'Slight' : [np.nan], 'Pr(Ai)': [np.nan]})

for region in df_1['LSOA_of_Accident_Location'].unique():
    x = items_df_preprocessed[df_1['Region'] == f'{region}'].groupby('Accident severity')['Accidents'].agg(sum).reset_index(name='Value count')
    y = np.array(x['Value count'])
  # y = np.insert(y, 0, 3, axis=0)
  # source: https://stackoverflow.com/a/58293212
    data_to_append = {}
    for i in range(len(Region_Severity_df.columns)):
        if(i>=1 and i !=4):
            data_to_append[Region_Severity_df.columns[i]] = y[i-1]
        elif (i == 4):
            data_to_append[Region_Severity_df.columns[i]] = sum(y)
            
    Region_Severity_df = Region_Severity_df.append(data_to_append, ignore_index = True)

Region_Severity_df = Region_Severity_df.drop(0,0)
Region_Severity_df['Region'] = RTAs_region_year_df['Region'].unique()
print(sum(Region_Severity_df['Pr(Ai)']))
sum_accidents = sum(Region_Severity_df['Pr(Ai)'])
Region_Severity_df